In [65]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler as StS
from xgboost import XGBClassifier as XGB
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LassoCV

In [2]:
#データ取り込み
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
#特徴量のラベルの日本語訳データ
label = pd.read_csv("column_labels.csv")

In [4]:
#特徴量ラベルを英語から日本語に変更(視覚的にわかりやすくするため)
eng_to_jpn = dict(zip(label["English"], label["Japanese"]))

train_jpn = train.rename(columns=eng_to_jpn)
test_jpn = test.rename(columns=eng_to_jpn)

In [5]:
train_jpn.head()

,ID,建物クラス,ゾーニング（用途地域）,道路に面する距離,敷地面積,接道の種類,路地の種類,区画形状,地勢,インフラ整備状況,...,プール面積,プールの品質,フェンスの種類,その他の特徴,その他特徴の価値,販売月,販売年,販売タイプ,販売条件,販売価格
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
train_jpn.describe()

,ID,建物クラス,道路に面する距離,敷地面積,全体的な品質評価,全体的な状態評価,建築年,改築年,石材仕上げの面積,地下室仕上げ面積1,...,ウッドデッキ面積,オープンポーチ面積,囲いポーチ面積,3シーズンポーチ面積,スクリーンポーチ面積,プール面積,その他特徴の価値,販売月,販売年,販売価格
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
train_jpn.columns.tolist()

['ID',
 '建物クラス',
 'ゾーニング（用途地域）',
 '道路に面する距離',
 '敷地面積',
 '接道の種類',
 '路地の種類',
 '区画形状',
 '地勢',
 'インフラ整備状況',
 '区画構成',
 '土地の傾斜',
 '地区',
 '主要道路等との近接状況1',
 '主要道路等との近接状況2',
 '住宅のタイプ',
 '住宅スタイル',
 '全体的な品質評価',
 '全体的な状態評価',
 '建築年',
 '改築年',
 '屋根のスタイル',
 '屋根の素材',
 '外装仕上げ1',
 '外装仕上げ2',
 '石材仕上げの種類',
 '石材仕上げの面積',
 '外装の品質',
 '外装の状態',
 '基礎の種類',
 '地下室の高さ評価',
 '地下室の状態評価',
 '地下室の外部への開放度',
 '地下室の仕上げタイプ1',
 '地下室仕上げ面積1',
 '地下室の仕上げタイプ2',
 '地下室仕上げ面積2',
 '未仕上げ地下面積',
 '地下室総面積',
 '暖房の種類',
 '暖房の品質と状態',
 'セントラル空調の有無',
 '電気システムの種類',
 '1階の面積',
 '2階の面積',
 '低品質仕上げ面積',
 '地上居住面積',
 '地下のフルバス数',
 '地下の半バス数',
 'フルバス数',
 '半バス数',
 '地上階の寝室数',
 '地上階のキッチン数',
 'キッチンの品質',
 '地上階の部屋数',
 '住宅の機能性',
 '暖炉数',
 '暖炉の品質',
 'ガレージタイプ',
 'ガレージ建築年',
 'ガレージの仕上げ',
 'ガレージ収容台数',
 'ガレージ面積',
 'ガレージの品質',
 'ガレージの状態',
 '舗装された私道の有無',
 'ウッドデッキ面積',
 'オープンポーチ面積',
 '囲いポーチ面積',
 '3シーズンポーチ面積',
 'スクリーンポーチ面積',
 'プール面積',
 'プールの品質',
 'フェンスの種類',
 'その他の特徴',
 'その他特徴の価値',
 '販売月',
 '販売年',
 '販売タイプ',
 '販売条件',
 '販売価格']

In [8]:
#欠損値補完　[数値→中央値、文字列やカテゴリ→"missing"]　それぞれ補完 
numeric_cols = train_jpn.select_dtypes(include="number").columns
train_jpn[numeric_cols] = train_jpn[numeric_cols].apply(lambda col: col.fillna(col.median()))

numeric_cols = test_jpn.select_dtypes(include="number").columns
test_jpn[numeric_cols] = test_jpn[numeric_cols].apply(lambda col: col.fillna(col.median()))

cat_cols = train_jpn.select_dtypes(include=["object","category"]).columns
train_jpn[cat_cols] = train_jpn[cat_cols].apply(lambda col: col.fillna("missing"))

cat_cols = test_jpn.select_dtypes(include=["object","category"]).columns
test_jpn[cat_cols] = test_jpn[cat_cols].apply(lambda col: col.fillna("missing"))

In [9]:
trainX = train_jpn.drop(["販売価格"],axis=1)
y = train_jpn["販売価格"]
testX = test_jpn.copy()

In [10]:
trainX["ガレージの状態"].value_counts()

ガレージの状態
TA         1326
missing      81
Fa           35
Gd            9
Po            7
Ex            2
Name: count, dtype: int64

In [11]:
trainX["プールの品質"].value_counts()

プールの品質
missing    1453
Gd            3
Ex            2
Fa            2
Name: count, dtype: int64

In [12]:
#カテゴリ変数系の特徴量のリスト化
quality = ["全体的な品質評価","全体的な状態評価","外装の品質","外装の状態","地下室の高さ評価","地下室の状態評価","暖房の品質と状態"
            ,"キッチンの品質","暖炉の品質","ガレージの品質","ガレージの状態","プールの品質","その他特徴の価値"]

In [13]:
#カテゴリ特徴量の値の分布確認
for col in quality:
    print(f"\n★ {col} の値の分布")
    print(trainX[col].value_counts(dropna=False))


★ 全体的な品質評価 の値の分布
全体的な品質評価
5     397
6     374
7     319
8     168
4     116
9      43
3      20
10     18
2       3
1       2
Name: count, dtype: int64

★ 全体的な状態評価 の値の分布
全体的な状態評価
5    821
6    252
7    205
8     72
4     57
3     25
9     22
2      5
1      1
Name: count, dtype: int64

★ 外装の品質 の値の分布
外装の品質
TA    906
Gd    488
Ex     52
Fa     14
Name: count, dtype: int64

★ 外装の状態 の値の分布
外装の状態
TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: count, dtype: int64

★ 地下室の高さ評価 の値の分布
地下室の高さ評価
TA         649
Gd         618
Ex         121
missing     37
Fa          35
Name: count, dtype: int64

★ 地下室の状態評価 の値の分布
地下室の状態評価
TA         1311
Gd           65
Fa           45
missing      37
Po            2
Name: count, dtype: int64

★ 暖房の品質と状態 の値の分布
暖房の品質と状態
Ex    741
TA    428
Gd    241
Fa     49
Po      1
Name: count, dtype: int64

★ キッチンの品質 の値の分布
キッチンの品質
TA    735
Gd    586
Ex    100
Fa     39
Name: count, dtype: int64

★ 暖炉の品質 の値の分布
暖炉の品質
missing    690
Gd         380
TA         313
Fa 

In [14]:
for col in quality:
    print(f"\n★【testX】{col} の値の分布")
    print(testX[col].value_counts(dropna=False))


★【testX】全体的な品質評価 の値の分布
全体的な品質評価
5     428
6     357
7     281
8     174
4     110
9      64
3      20
10     13
2      10
1       2
Name: count, dtype: int64

★【testX】全体的な状態評価 の値の分布
全体的な状態評価
5    824
6    279
7    185
8     72
4     44
3     25
9     19
1      6
2      5
Name: count, dtype: int64

★【testX】外装の品質 の値の分布
外装の品質
TA    892
Gd    491
Ex     55
Fa     21
Name: count, dtype: int64

★【testX】外装の状態 の値の分布
外装の状態
TA    1256
Gd     153
Fa      39
Ex       9
Po       2
Name: count, dtype: int64

★【testX】地下室の高さ評価 の値の分布
地下室の高さ評価
TA         634
Gd         591
Ex         137
Fa          53
missing     44
Name: count, dtype: int64

★【testX】地下室の状態評価 の値の分布
地下室の状態評価
TA         1295
Fa           59
Gd           57
missing      45
Po            3
Name: count, dtype: int64

★【testX】暖房の品質と状態 の値の分布
暖房の品質と状態
Ex    752
TA    429
Gd    233
Fa     43
Po      2
Name: count, dtype: int64

★【testX】キッチンの品質 の値の分布
キッチンの品質
TA         757
Gd         565
Ex         105
Fa          31
missing      1
Name: count,

In [15]:
#数値の特徴量名と大半が欠損値の特徴量名を除いてリスト化する
quality_cols = ["外装の品質","外装の状態","地下室の高さ評価","地下室の状態評価","暖房の品質と状態"
                ,"キッチンの品質","暖炉の品質","ガレージの品質","ガレージの状態"]

#大半が欠損値の"プールの品質"特徴量を削除
trainX = trainX.drop(["プールの品質"],axis=1)
testX = testX.drop(["プールの品質"],axis=1)

In [16]:
#カテゴリ変数特徴量を順序エンコーディング
for col in quality_cols:
    trainX[col] = trainX[col].map({"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"NA":0,"missing":0})

for col in quality_cols:
    testX[col] = testX[col].map({"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"NA":0,"missing":0})

In [17]:
#object特徴量名をリスト化
cat_cols = trainX.select_dtypes(include=["object"]).columns

In [18]:
#object特徴量をターゲットエンコーディング
enc_cols_train = {}
enc_cols_test = {}

for col in cat_cols:
    means = train_jpn.groupby(col)["販売価格"].mean()
    enc_cols_train[col + '_enc'] = trainX[col].map(means)
    enc_cols_test[col + '_enc'] = testX[col].map(means)

trainX = pd.concat([trainX, pd.DataFrame(enc_cols_train)], axis=1)
testX = pd.concat([testX, pd.DataFrame(enc_cols_test)], axis=1)

In [19]:
#object特徴量を削除
trainX_enc = trainX.drop(cat_cols,axis=1)
testX_enc = testX.drop(cat_cols,axis=1)

In [20]:
#面積についての特徴量名をリスト化
area_cols = ['敷地面積','石材仕上げの面積','地下室仕上げ面積1','地下室仕上げ面積2','未仕上げ地下面積','地下室総面積','1階の面積','2階の面積',
'低品質仕上げ面積','地上居住面積','地上居住面積','ガレージ面積','ウッドデッキ面積','オープンポーチ面積','3シーズンポーチ面積','スクリーンポーチ面積',
 'プール面積']

In [21]:
trainX_enc[area_cols].describe()

,敷地面積,石材仕上げの面積,地下室仕上げ面積1,地下室仕上げ面積2,未仕上げ地下面積,地下室総面積,1階の面積,2階の面積,低品質仕上げ面積,地上居住面積,地上居住面積,ガレージ面積,ウッドデッキ面積,オープンポーチ面積,3シーズンポーチ面積,スクリーンポーチ面積,プール面積
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,103.117123,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,1515.463699,472.980137,94.244521,46.660274,3.409589,15.060959,2.758904
std,9981.264932,180.731373,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,525.480383,213.804841,125.338794,66.256028,29.317331,55.757415,40.177307
min,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7553.500000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,1129.500000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9478.500000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,1464.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000
75%,11601.500000,164.250000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1776.750000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000
max,215245.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,5642.000000,1418.000000,857.000000,547.000000,508.000000,480.000000,738.000000


In [22]:
#0の割合を出力
for col in area_cols:
    zero_ratio = (trainX_enc[col] == 0).mean()
    print(f'{col}: {zero_ratio:.2%} が 0')

敷地面積: 0.00% が 0
石材仕上げの面積: 59.52% が 0
地下室仕上げ面積1: 31.99% が 0
地下室仕上げ面積2: 88.56% が 0
未仕上げ地下面積: 8.08% が 0
地下室総面積: 2.53% が 0
1階の面積: 0.00% が 0
2階の面積: 56.78% が 0
低品質仕上げ面積: 98.22% が 0
地上居住面積: 0.00% が 0
地上居住面積: 0.00% が 0
ガレージ面積: 5.55% が 0
ウッドデッキ面積: 52.12% が 0
オープンポーチ面積: 44.93% が 0
3シーズンポーチ面積: 98.36% が 0
スクリーンポーチ面積: 92.05% が 0
プール面積: 99.52% が 0


In [23]:
for col in area_cols:
    zero_ratio = (testX_enc[col] == 0).mean()
    print(f'{col}: {zero_ratio:.2%} が 0')

敷地面積: 0.00% が 0
石材仕上げの面積: 61.14% が 0
地下室仕上げ面積1: 31.67% が 0
地下室仕上げ面積2: 87.66% が 0
未仕上げ地下面積: 8.43% が 0
地下室総面積: 2.81% が 0
1階の面積: 0.00% が 0
2階の面積: 57.51% が 0
低品質仕上げ面積: 99.04% が 0
地上居住面積: 0.00% が 0
地上居住面積: 0.00% が 0
ガレージ面積: 5.21% が 0
ウッドデッキ面積: 52.23% が 0
オープンポーチ面積: 44.00% が 0
3シーズンポーチ面積: 99.11% が 0
スクリーンポーチ面積: 90.40% が 0
プール面積: 99.59% が 0


In [24]:
#0が80％以上の特徴量について0,1に変えて、あるかないかという特徴量に変換（バイナリ化）
sparse_cols = []
for col in area_cols:
    zero_ratio = (trainX_enc[col] == 0).mean()
    if zero_ratio >= 0.8:
        new_col = f'{col}_flag'
        trainX_enc[new_col] = trainX_enc[col].apply(lambda x: 1 if x > 0 else 0)
        testX_enc[new_col] = testX_enc[col].apply(lambda x: 1 if x > 0 else 0)
        sparse_cols.append(col)

In [25]:
trainX_bin = trainX_enc.drop(sparse_cols,axis=1)
testX_bin = testX_enc.drop(sparse_cols,axis=1)

In [26]:
#改築済みかどうかを特徴量化
trainX_bin['改築済み'] = (trainX_bin['建築年'] != trainX_bin['改築年']).astype(int)
testX_bin['改築済み'] = (testX_bin['建築年'] != testX_bin['改築年']).astype(int)

In [27]:
#建築年数、改築年数、改築が10年以内のものをそれぞれ特徴量化
trainX_bin['建築年数'] = trainX_bin['販売年'] - trainX_bin['建築年']

trainX_bin['改築年数'] = trainX_bin['販売年'] - trainX_bin['改築年']

trainX_bin['最近改築した'] = (trainX_bin['改築年数'] <= 10).astype(int)

testX_bin['建築年数'] = testX_bin['販売年'] - testX_bin['建築年']

testX_bin['改築年数'] = testX_bin['販売年'] - testX_bin['改築年']

testX_bin['最近改築した'] = (testX_bin['改築年数'] <= 10).astype(int)

In [28]:
#建築年、改築年特徴量を削除
trainX_bin_plus = trainX_bin.drop(["建築年","改築年"],axis=1)
testX_bin_plus = testX_bin.drop(["建築年","改築年"],axis=1)

In [29]:
missing = trainX_bin_plus.isnull().sum()
missing_cols = missing[missing > 0]
print(missing_cols)

Series([], dtype: int64)


In [30]:
missing = testX_bin_plus.isnull().sum()
missing_cols = missing[missing > 0].index.tolist()
print(missing_cols)

['ゾーニング（用途地域）_enc', 'インフラ整備状況_enc', '外装仕上げ1_enc', '外装仕上げ2_enc', '住宅の機能性_enc', '販売タイプ_enc']


In [31]:
testX_bin_plus[missing_cols].describe()

,ゾーニング（用途地域）_enc,インフラ整備状況_enc,外装仕上げ1_enc,外装仕上げ2_enc,住宅の機能性_enc,販売タイプ_enc
count,1455.000000,1.457000e+03,1458.000000,1458.000000,1457.000000,1458.000000
mean,179806.722628,1.809510e+05,180127.410665,180631.240643,180983.479537,180526.668551
std,27607.779632,2.911382e-09,31496.076252,31190.379777,9916.660452,29252.071128
min,74528.000000,1.809510e+05,71000.000000,105000.000000,85800.000000,119850.000000
25%,191004.994787,1.809510e+05,149841.645631,149803.172897,183429.147059,173401.836622
50%,191004.994787,1.809510e+05,163077.450450,168112.387324,183429.147059,173401.836622
75%,191004.994787,1.809510e+05,213732.900971,214432.460317,183429.147059,173401.836622
max,214014.061538,1.809510e+05,231690.655738,252070.000000,183429.147059,274945.418033


In [83]:
#testデータの欠損値を中央値で補完
for col in missing_cols:
    testX_bin_plus[col] = testX_bin_plus[col].fillna(testX_bin_plus[col].median())

In [69]:
from sklearn.ensemble import GradientBoostingRegressor
model_GBR = GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    random_state=42
)

In [71]:
model_GBR.fit(trainX_bin_plus,y)

GradientBoostingRegressor(max_depth=4, n_estimators=150, random_state=42)

In [89]:
pred_GBR = model_GBR.predict(testX_bin_plus)

In [105]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor()
param_grid =  {'max_depth': [3, 5,7],'learning_rate': [0.05,0.1, 0.3],'n_estimators': [50, 100,200]}
gcv = GridSearchCV(estimator=xgb_model, param_grid=param_grid,scoring='neg_root_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

In [107]:
gcv.fit(trainX_bin_plus,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.3],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [109]:
pred_xgb = gcv.predict(testX_bin_plus)

In [99]:
import lightgbm as lgb
model_LR = lgb.LGBMRegressor(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    random_state=42
)

In [101]:
model_LR.fit(trainX_bin_plus,y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000775 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3588
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 77
[LightGBM] [Info] Start training from score 180921.195890
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

LGBMRegressor(max_depth=4, n_estimators=150, random_state=42)

In [103]:
pred_LR = model_LR.predict(testX_bin_plus)

In [111]:
from catboost import CatBoostRegressor
model_CBR = CatBoostRegressor(iterations=1000,learning_rate=0.05,depth=4,random_seed=42,verbose=0)

In [113]:
model_CBR.fit(trainX_bin_plus,y)

In [115]:
pred_cbr = model_CBR.predict(testX_bin_plus)

In [119]:
#それそれのモデルのLBでの結果から割合を決めアンサンブル
ensemble_pred = (pred_GBR * 0.2486 + pred_LR * 0.2491 + pred_cbr * 0.2547 + pred_xgb * 0.2476)

In [121]:
sample["SalePrice"] = ensemble_pred
sample.to_csv("submission_ens.csv",index=None)